#TASK 1

In [ ]:
!pip install langchain langchain-community chromadb sentence-transformers torch feedparser lxml tiktoken pandas

In [ ]:
# 1. INSTALACIÓN E IMPORTACIONES

import feedparser # Requerido por RSSFeedLoader
import tiktoken
import pandas as pd
from typing import List

# Componentes de LangChain

from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema.document import Document
from langchain.schema.retriever import BaseRetriever

print("Librerías importadas correctamente.")

Librerías importadas correctamente.


In [ ]:
# 2. CONFIGURACIÓN Y CONSTANTES

RSS_URL = "https://rpp.pe/rss"
MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
COLLECTION_NAME = "rpp_news_v1"

In [ ]:
# 3. PIPELINE MODULAR (Cumpliendo Paso 5: Orquestación)

# -----------------------------------------------------------------------
# PASO 0: Carga de Datos (Load Data)
# -----------------------------------------------------------------------
def load_rpp_news(url: str, limit: int = 50) -> List[Document]:
    """
    Carga las últimas 'limit' noticias del feed RSS de RPP.
    Usa 'feedparser' directamente para evitar la dependencia 'newspaper3k'.
    """
    print(f"Cargando {limit} noticias desde {url} usando feedparser...")

    # 1. Parsear el feed
    feed = feedparser.parse(url)

    # 2. Verificar si hay entradas
    if not feed.entries:
        print("Error: El feed no contiene entradas (entries).")
        return []

    docs = []
    # 3. Iterar sobre las entradas hasta el límite
    for entry in feed.entries[:limit]:

        # El contenido principal es el 'summary' o 'description' en RSS
        page_content = entry.get("summary", entry.get("description", ""))

        # Recolectar los metadatos solicitados
        metadata = {
            "title": entry.get("title", "N/A"),
            "link": entry.get("link", "N/A"),
            # 'published' a veces viene en 'published_parsed'
            "published": entry.get("published", "N/A")
        }

        # 4. Crear el Documento de LangChain
        # Este formato (page_content + metadata) es el que
        # el resto del pipeline (chunking, embedding) espera.
        doc = Document(page_content=page_content, metadata=metadata)
        docs.append(doc)

    print(f"Se cargaron {len(docs)} documentos exitosamente.")
    return docs

# -----------------------------------------------------------------------
# PASO 1: Tokenización (Tokenization)
# -----------------------------------------------------------------------
def analyze_tokenization(sample_text: str):
    """
    Analiza un texto de muestra con tiktoken para decidir si se necesita chunking.
    """
    print("\n--- Análisis de Tokenización (Paso 1) ---")
    # Usamos cl100k_base, el encoding estándar para modelos como GPT-3.5/4
    encoding = tiktoken.get_encoding("cl100k_base")

    tokens = encoding.encode(sample_text)
    num_tokens = len(tokens)

    print(f"Texto de muestra: \"{sample_text[:100]}...\"")
    print(f"Número de tokens (cl100k_base): {num_tokens}")

    # Límite de contexto de 'all-MiniLM-L6-v2' es 512 tokens.
    # Los RSS 'description' suelen ser cortos, pero es buena práctica.
    if num_tokens > 400: # Dejamos un margen
        print("Decisión: El texto es largo. Se recomienda 'chunking'.")
    else:
        print("Decisión: El texto es corto. 'Chunking' no es estrictamente necesario,")
        print("          pero se aplicará para mantener consistencia en el pipeline.")
    print("---------------------------------------------")


def split_documents(docs: List[Document]) -> List[Document]:
    """
    Divide los documentos en chunks más pequeños usando un token splitter.
    """
    # Usamos un text splitter de LangChain.
    # Mide la longitud usando tiktoken (cl100k_base) por defecto.
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=512,  # Tamaño máximo de chunk (alineado con el modelo de embedding)
        chunk_overlap=50   # Superposición para mantener contexto entre chunks
    )

    print(f"\nDividiendo {len(docs)} documentos en chunks...")
    splits = text_splitter.split_documents(docs)
    print(f"Se generaron {len(splits)} chunks.")
    return splits

# -----------------------------------------------------------------------
# PASO 2: Embedding
# -----------------------------------------------------------------------
def get_embedding_model(model_name: str) -> HuggingFaceEmbeddings:
    """
    Inicializa el modelo de embeddings de SentenceTransformers.
    """
    print(f"\nCargando modelo de embeddings: {model_name}...")

    model_kwargs = {'device': 'cpu'}
    encode_kwargs = {'normalize_embeddings': False}

    embeddings = HuggingFaceEmbeddings(
        model_name=model_name,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs
    )
    return embeddings

# -----------------------------------------------------------------------
# PASO 3: Creación de Chroma Collection y Retriever
# -----------------------------------------------------------------------
def create_vectorstore_and_retriever(splits: List[Document],
                                     embedding_model: HuggingFaceEmbeddings,
                                     collection_name: str) -> BaseRetriever:
    """
    Crea la base de datos vectorial (ChromaDB) y el retriever.
    Usamos una base de datos en memoria (persist_directory=None).
    """
    print(f"Creando vector store en memoria (ChromaDB) con {len(splits)} chunks...")

    vectorstore = Chroma.from_documents(
        documents=splits,
        embedding=embedding_model,
        collection_name=collection_name

    )

    print("Vector store creado.")

    # Implementa el retriever con búsqueda de similitud (k=5 mejores resultados)
    retriever = vectorstore.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 5}
    )
    return retriever

# -----------------------------------------------------------------------
# PASO 4: Consulta de Resultados (Query Results)
# -----------------------------------------------------------------------
def query_and_format_results(retriever: BaseRetriever, query: str) -> pd.DataFrame:
    """
    Realiza una consulta al retriever y formatea los resultados en un DataFrame.
    """
    print(f"\nEjecutando consulta: \"{query}\"")
    retrieved_docs = retriever.invoke(query)

    print(f"Resultados obtenidos: {len(retrieved_docs)}")

    # Formateamos los resultados según lo solicitado
    data = []
    for doc in retrieved_docs:
        data.append({
            "title": doc.metadata.get("title", "N/A"),
            "description": doc.page_content, # El contenido del chunk recuperado
            "link": doc.metadata.get("link", "N/A"),
            # El formato de fecha de 'published' puede variar, ajustamos
            "date_published": doc.metadata.get("published", "N/A")
        })

    # Convertimos a DataFrame
    df = pd.DataFrame(data)
    return df

In [ ]:

# 4. EJECUCIÓN PRINCIPAL (Orquestación End-to-End)

if __name__ == "__main__":

    # --------------------------------
    # PASO 0: Carga de Datos
    # --------------------------------
    documents = load_rpp_news(RSS_URL, limit=50)

    if documents:
        # --------------------------------
        # PASO 1: Tokenización (Análisis)
        # --------------------------------
        # Tomamos el 'page_content' de la primera noticia para el análisis
        sample_article_text = documents[0].page_content
        analyze_tokenization(sample_article_text)

        # Aplicamos el splitting (parte del pipeline)
        doc_splits = split_documents(documents)

        # --------------------------------
        # PASO 2: Embedding
        # --------------------------------
        embedding_model = get_embedding_model(MODEL_NAME)

        # --------------------------------
        # PASO 3: Chroma Collection y Retriever
        # --------------------------------
        # (Upsert) Chroma.from_documents crea o actualiza la colección
        rpp_retriever = create_vectorstore_and_retriever(
            doc_splits,
            embedding_model,
            COLLECTION_NAME
        )

        # --------------------------------
        # PASO 4: Consulta de Resultados
        # --------------------------------
        query = "Últimas noticias de economía"
        results_df = query_and_format_results(rpp_retriever, query)

        # Mostramos los resultados en formato DataFrame
        print("\n--- Resultados de la Búsqueda (DataFrame) ---")
        pd.set_option('display.max_colwidth', 100) # Para ver mejor las descripciones
        print(results_df)

    else:
        print("No se pudieron cargar noticias del feed RSS.")

Cargando 50 noticias desde https://rpp.pe/rss usando feedparser...
Se cargaron 50 documentos exitosamente.

--- Análisis de Tokenización (Paso 1) ---
Texto de muestra: "El abogado, quien además representa al presidente de la Junta Nacional de Justicia (JNJ), Gino Ríos,..."
Número de tokens (cl100k_base): 83
Decisión: El texto es corto. 'Chunking' no es estrictamente necesario,
          pero se aplicará para mantener consistencia en el pipeline.
---------------------------------------------

Dividiendo 50 documentos en chunks...
Se generaron 50 chunks.

Cargando modelo de embeddings: sentence-transformers/all-MiniLM-L6-v2...


/tmp/ipython-input-3090899392.py:100: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Creando vector store en memoria (ChromaDB) con 50 chunks...
Vector store creado.

Ejecutando consulta: "Últimas noticias de economía"
Resultados obtenidos: 5

--- Resultados de la Búsqueda (DataFrame) ---
                                                                                                 title  \
0  Isabel Preysler publica íntimas cartas de amor de Mario Vargas Llosa y afirma: "Él era feliz con...   
1      Estados Unidos anunciará un "aumento sustancial" en sus sanciones a Rusia en las próximas horas   
2  'Emily in Paris' llega a Italia: Lily Collins conquistará Roma en la temporada 5 de la serie de ...   
3         ¡Ya es madre! Valeria Flórez dio a luz a su primer hijo y emociona a todos en redes sociales   
4  Julio Rodríguez explica los alcances de la resolución de la JNJ que mantiene la suspensión de De...   

                                                                                           description  \
0  Isabel Preysler publicó su libro 'Mi verdadera hi

In [ ]:
results_df

,title,description,link,date_published
0,"Isabel Preysler publica íntimas cartas de amor de Mario Vargas Llosa y afirma: ""Él era feliz con...",Isabel Preysler publicó su libro 'Mi verdadera historia' donde aparecen cartas de amor que le en...,https://rpp.pe/famosos/celebridades/isabel-preysler-publica-intimas-cartas-de-amor-de-mario-varg...,"Wed, 22 Oct 2025 08:47:59 -0500"
1,"Estados Unidos anunciará un ""aumento sustancial"" en sus sanciones a Rusia en las próximas horas","""Vamos a anunciar después del cierre (de los mercados) de esta tarde o a primera hora de mañana ...",https://rpp.pe/mundo/estados-unidos/estados-unidos-anunciara-un-aumento-sustancial-en-sus-sancio...,"Wed, 22 Oct 2025 16:33:24 -0500"
2,'Emily in Paris' llega a Italia: Lily Collins conquistará Roma en la temporada 5 de la serie de ...,La protagonista inicia una nueva etapa profesional y amorosa en Italia en una temporada que prom...,https://rpp.pe/tv/netflix/emily-in-paris-llega-a-italia-lily-collins-conquistara-roma-en-la-temp...,"Wed, 22 Oct 2025 16:31:14 -0500"
3,¡Ya es madre! Valeria Flórez dio a luz a su primer hijo y emociona a todos en redes sociales,"Tatiana Calmell, Camila Escribens, entre otras reinas de belleza, felicitaron a la modelo Valeri...",https://rpp.pe/famosos/farandula/valeria-florez-se-convierte-en-madre-por-primera-vez-y-presenta...,"Wed, 22 Oct 2025 17:14:58 -0500"
4,Julio Rodríguez explica los alcances de la resolución de la JNJ que mantiene la suspensión de De...,"El abogado, quien además representa al presidente de la Junta Nacional de Justicia (JNJ), Gino R...",https://rpp.pe/politica/judiciales/julio-rodriguez-explica-los-alcances-de-la-resolucion-de-la-j...,"Wed, 22 Oct 2025 18:27:32 -0500"
